In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from  sklearn.calibration import CalibratedClassifierCV

from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
train_df = pd.read_csv("train.csv")
train_df = train_df.drop(columns=['id'])
train_df.head()

,url,title,target
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


В экспериментах установил, что предобрабатывать title для count- и tf-idf- vectorizer не нужно, тк результаты ухудшаются до 80% (вероятно связано с тем, что в данной задаче очень важны окончания и сами словосочетания как они есть в оригинале), но при этом небольшая корректировка url пойдет на пользу. Я решил разделить на слова + удалить некоторые бесполезные приставки по типу 'www' и 'm'.

In [ ]:
def prepare_url(url):
    url_list = url.split('.')
    url_list = ('-'.join(url_list)).split('-')
    
    useless_substring = ['www', 'd', 'm', 'http', 'https', 'spb']
    for elem in url_list:
        if elem in useless_substring:
            url_list.remove(elem)
    
    
    return ' '.join(url_list)

train_df.loc[:, 'url'] = train_df.url.apply(prepare_url)
train_df.head()

,url,title,target
0,kp md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,kp by,Эта песня стала известна многим телезрителям б...,False
2,fanserials tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,colorbox ru,Не Беси Меня Картинки,False
4,tula sport ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [ ]:
from sklearn.base import TransformerMixin


class ColumnExtractor(TransformerMixin):
    
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.column_name].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df[['url', 'title']], 
                                                    train_df['target'].astype(int), 
                                                    test_size=0.25, shuffle=False)

Выбор n-грамм тоже осознанный: для url я посчитал, что необходимо анализировать посимвольно, так как там могут возникнуть интересные словечки, причем их длина будет лежат от 4 до 7 символов: porn, sexxx и другие возможные варианты

Для title оказалось лучше всего смотреть n-граммы слов, причем я ищу либо сами слова, либо словосочетание из 2-х, что дает наибольший окрас

In [ ]:
pipeline = Pipeline([
    (
        'features', 
        FeatureUnion([
            (
                'url', 
                Pipeline([
                    ('extractor', ColumnExtractor('url')),
                    (
                        'vectorizer', 
                        TfidfVectorizer(lowercase=True, ngram_range=(4, 7), analyzer='char')
                    )
                ])
            ),
            (
                'title',
                Pipeline([
                    ('extractor', ColumnExtractor('title')),
                    (
                        'vectorizer', 
                        TfidfVectorizer(lowercase=True, ngram_range=(1, 2))
                    )
                ])
            )
        ])
    ),
    #('clf', LogisticRegression(random_state=42, solver='liblinear', max_iter=1000, C=10))
    ('clf', SGDClassifier(random_state=42, loss='log', class_weight='balanced', alpha=0.000001))
    #('clf', Perceptron(penalty='l2', random_state=42, class_weight='balanced', alpha=0.000001, tol=1e-4, ))
])

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('url',
                                                 Pipeline(steps=[('extractor',
                                                                  <__main__.ColumnExtractor object at 0x1C2CE390>),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(analyzer='char',
                                                                                  ngram_range=(4,
                                                                                               7)))])),
                                                ('title',
                                                 Pipeline(steps=[('extractor',
                                                                  <__main__.ColumnExtractor object at 0x1C2CE330>),
                                                                 ('vectorizer',
      

In [ ]:
y_pred = pipeline.predict(X_train)
f1_score(y_train, y_pred)

0.9994398655677362

In [ ]:
y_test_pred = pipeline.predict(X_test)
f1_score(y_test, y_test_pred)

0.9815034384633625

Я применял различные модели, однако стохастический градиентный спуск с log функцией потерь дал невероятый результат по сравнению с другими линейными моделями (да и деревьями тоже). Также он показал лучший результат даже среди ансамблей (но тут я вероятно облажался и не смог подобрать лучший классификатор). Вариант применения бустинга я вытащил в отдельный файл, там пробовал много чего, но результат всегда был хуже нежели используемый здесь выбор

Используем лучший результат для тестовой выборки

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [ ]:
#for i in range(test_df.shape[0]):
#    test_df.loc[i, 'url'] = prepare_url(test_df['url'][i])
test_df.loc[:, 'url'] = test_df.url.apply(prepare_url)
test_df.head()

,id,url,title
0,135309,kommersant ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha ru,Женщинам | Империя Меха - Part 12
3,135312,national porn com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis ru,67


In [ ]:
X_test = test_df[["url", "title"]]

In [ ]:
test_df["target"] = pipeline.predict(X_test).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline_ver5.csv", index=False)

In [ ]:
!cat ml_baseline.csv | head

id,target

cat: write error: No space left on device



135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
